In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [210]:
df = pd.read_excel('Telco_customer_churn_adapted_v2.xlsx')

In [211]:
df.drop(columns=['Customer ID'], inplace=True)

In [212]:
# drop latitude dan longitude
df.drop(['Latitude','Longitude'], axis=1, inplace=True)

In [213]:
# rename Monthly Purchase (Thou. IDR) menjadi Monthly Purchase dan CLTV (Predicted Thou. IDR) menjadi CLTV
df.rename(columns={'Monthly Purchase (Thou. IDR)':'Monthly Purchase', 'CLTV (Predicted Thou. IDR)':'CLTV'}, inplace=True)

In [214]:
# ubah cltv dan monthly purchase dengan dikali 1000
df['CLTV'] = df['CLTV']*1000
df['Monthly Purchase'] = df['Monthly Purchase']*1000

In [208]:
# ubah churn label yes ke 1 dan no ke 0
df['Churn Label'] = df['Churn Label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [122]:
# buat kolom no internet service jika minimal salah satu diantara 'Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp' memiliki nilai no internet service
df['No Internet Service'] = df[['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']].apply(lambda x: 1 if 'No internet service' in x.values else 0, axis=1)

In [123]:
# ubah no internet service menjadi no
df[['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']] = df[['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']].replace('No internet service', 'No')

In [124]:
# ubah nilai menjadi 1 jika ada yes dan 0 jika ada no
df[['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']] = df[['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']].apply(lambda x: x.map({'Yes':1, 'No':0}))

In [215]:
service = ['Games Product', 'Music Product', 'Education Product', 'Call Center','Video Product', 'Use MyApp']

# buat variabel yang berisi jumlah service yang berisi 'Yes' dari variabel service
# Define the function to calculate the total number of 'Yes' in the service-related columns
def calculate_total_service(row):
    services = ['Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp']
    return sum(row[service] == 'Yes' for service in services)

# Apply the function across the dfframe
df['Total Service'] = df.apply(calculate_total_service, axis=1)

In [216]:
# bagi cltv dengan montly purchase
df['CLTV per Purchase'] = df['CLTV'] / df['Monthly Purchase']


# Clustering

In [217]:
# cluster dengan k means berdasarkan cltv, monthly purchase, total service
from sklearn.cluster import KMeans

# gunakan 5 cluster
km = KMeans(n_clusters=5)

# standard scales 'CLTV', 'Monthly Purchase', 'Total Service' dan buat df_scale 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scale = scaler.fit_transform(df[['CLTV', 'Monthly Purchase', 'Total Service']])

# fit data
km.fit(df_scale)

# tambahkan hasil cluster ke df_cluster sebagai kolom baru
df['Cluster'] = km.labels_

d:\Arsip Kompetisi\Data Challenge DSW 2023\dsw\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [171]:
# export km
import pickle

pickle.dump(scaler, open('scaler.pkl', 'wb'))
pickle.dump(km, open('cluster.pkl', 'wb'))

# Training model

In [218]:
# ambil hanya churn label 1
df_churn = df[df['Churn Label']=='Yes']

In [196]:
X.columns

Index(['Location', 'Device Class', 'Games Product', 'Music Product',
       'Education Product', 'Call Center', 'Video Product', 'Use MyApp',
       'Payment Method', 'Monthly Purchase', 'CLTV', 'Total Service',
       'CLTV per Purchase', 'Cluster'],
      dtype='object')

In [ ]:
# # map service 0 ke no dan 1 ke yes
# df_churn['Games Product'] = df_churn['Games Product'].map({0:'No', 1:'Yes'})
# df_churn['Music Product'] = df_churn['Music Product'].map({0:'No', 1:'Yes'})
# df_churn['Education Product'] = df_churn['Education Product'].map({0:'No', 1:'Yes'})
# df_churn['Call Center'] = df_churn['Call Center'].map({0:'No', 1:'Yes'})
# df_churn['Video Product'] = df_churn['Video Product'].map({0:'No', 1:'Yes'})
# df_churn['Use MyApp'] = df_churn['Use MyApp'].map({0:'No', 1:'Yes'})

In [186]:
df_churn.drop(columns=['Churn Label'], inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_5272\2198768239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_churn.drop(columns=['Churn Label'], inplace=True)


In [177]:
df_churn.columns

Index(['Tenure Months', 'Location', 'Device Class', 'Games Product',
       'Music Product', 'Education Product', 'Call Center', 'Video Product',
       'Use MyApp', 'Payment Method', 'Monthly Purchase', 'CLTV',
       'Total Service', 'CLTV per Purchase', 'Cluster'],
      dtype='object')

In [194]:
df_churn.to_csv('df_churn.csv', index=False)

In [187]:
# catboost untuk prediksi Tenure Months
from catboost import CatBoostRegressor

# cat features = 'Location', 'Device Class','Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp', 'Payment Method'
cat_features = ['Location', 'Games Product',
       'Music Product', 'Education Product', 'Call Center', 'Video Product',
       'Use MyApp', 'Cluster','Payment Method', 'Device Class']

# prediksi Tenure Months
model = CatBoostRegressor(cat_features=cat_features, random_state=2020,
                          l2_leaf_reg=10)

# split
from sklearn.model_selection import train_test_split

X = df_churn.drop(['Tenure Months'], axis=1)
y = df_churn['Tenure Months']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

# fit model
model.fit(X, y)

0:	learn: 19.3470612	total: 91.8ms	remaining: 1m 31s
1:	learn: 19.1680278	total: 137ms	remaining: 1m 8s
2:	learn: 18.9967426	total: 182ms	remaining: 1m
3:	learn: 18.8463189	total: 222ms	remaining: 55.2s
4:	learn: 18.6916658	total: 268ms	remaining: 53.4s
5:	learn: 18.5260319	total: 317ms	remaining: 52.5s
6:	learn: 18.3823719	total: 363ms	remaining: 51.6s
7:	learn: 18.2237277	total: 404ms	remaining: 50.1s
8:	learn: 18.0746936	total: 449ms	remaining: 49.5s
9:	learn: 17.9458247	total: 505ms	remaining: 50s
10:	learn: 17.8200696	total: 557ms	remaining: 50.1s
11:	learn: 17.7025739	total: 603ms	remaining: 49.6s
12:	learn: 17.5761658	total: 644ms	remaining: 48.9s
13:	learn: 17.4701964	total: 696ms	remaining: 49s
14:	learn: 17.3592631	total: 747ms	remaining: 49s
15:	learn: 17.2570872	total: 806ms	remaining: 49.6s
16:	learn: 17.1716905	total: 875ms	remaining: 50.6s
17:	learn: 17.0676999	total: 922ms	remaining: 50.3s
18:	learn: 16.9694924	total: 985ms	remaining: 50.9s
19:	learn: 16.8717854	total: 

In [190]:
# cek akurasi training
model.score(X, y)

0.5461936299362211

In [192]:
# evaluate model
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X)
mean_absolute_error(y, y_pred)
print('MAE:', mean_absolute_error(y, y_pred))

# r2 scores
from sklearn.metrics import r2_score
r2 = r2_score(y, y_pred)
print('R2:', r2)

MAE: 10.02626353691731
R2: 0.5461936299362211


In [181]:
model.save_model('model_cb')

In [193]:
# export model
import pickle

pickle.dump(model, open('model_cb.pkl', 'wb'))

In [222]:
# ambil hanya churn label 1
df_churn = df[df['Churn Label']=='Yes']

In [ ]:
df_churn_tenure = df_churn['Tenure Months']

df_churn = df_churn[['Location', 'Device Class', 'Games Product', 'Music Product', 'Education Product', 'Call Center', 'Video Product', 'Use MyApp', 'Payment Method', 'Monthly Purchase', 'CLTV', 'Total Service', 'CLTV per Purchase', 'Cluster']]

predictions = model.predict(df_churn)

df_churn['Predicted Tenure Months to Churn'] = predictions

df_churn['Predicted Tenure Months to Churn'] = df_churn['Predicted Tenure Months to Churn'].apply(lambda x: round(x))
            
# append df_churn dan df_churn_tenure
df_churn = pd.concat([df_churn, df_churn_tenure], axis=1) 

df_churn.sort_values(by=['Tenure Months'], inplace=True, ascending=True)
            # create index after sort
df_churn['Index'] = np.arange(len(df_churn))

            # show mae and r2 score and show using st.metric
mae = round(np.mean(abs(df_churn['Predicted Tenure Months to Churn'] - df_churn['Tenure Months'])), 2)
r2 = round(1 - (np.sum((df_churn['Tenure Months'] - df_churn['Predicted Tenure Months to Churn'])**2) / np.sum((df_churn['Tenure Months'] - np.mean(df_churn['Tenure Months']))**2)), 2)
mse = round(np.mean((df_churn['Tenure Months'] - df_churn['Predicted Tenure Months to Churn'])**2), 2)

In [231]:
df_churn.to_csv('df_churn.csv', index=False)

In [230]:
import plotly.express as px 

# buat line chart 'Predicted Tenure Months to Churn', 'Tenure Months' dengan plotly
fig = px.scatter(df_churn,x='Index', y=['Predicted Tenure Months to Churn', 'Tenure Months'],
                    title='Predicted Tenure Months to Churn vs Tenure Months')

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

# save plot
fig.write_html('plot.html')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('variable=Predicted Tenure Mont' ... '}<br>value=%{y}<extra></extra>'),
              'legendgroup': 'Predicted Tenure Months to Churn',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Predicted Tenure Months to Churn',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   0,    1,    2, ..., 1866, 1867, 1868]),
              'xaxis': 'x',
              'y': array([ 3,  5,  6, ..., 42, 65, 48], dtype=int64),
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Tenure Months<br>Index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Tenure Months',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Tenure Months',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([   0,    1,    2, ..., 1866, 1867, 1868]),
              'xaxis': 'x',
              'y': array([ 1,  1,  1, ..., 72, 72, 72], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'},
                          'tracegroupgap': 0,
                          'x': 0.01,
                          'xanchor': 'left',
                          'y': 0.99,
                          'yanchor': 'top'},
               'template': '...',
               'title': {'text': 'Predicted Tenure Months to Churn vs Tenure Months'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Index'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [234]:
# feature importances
importance = model.get_feature_importance(prettified=True)
importance

,Feature Id,Importances
0,Total Service,22.542275
1,CLTV,16.357402
2,Monthly Purchase,15.678679
3,Payment Method,11.668414
4,Music Product,8.814316
5,Cluster,6.944370
6,CLTV per Purchase,4.492681
7,Video Product,2.673589
8,Games Product,2.656629
9,Education Product,2.509702


In [161]:
df.drop(columns=['Tenure Months', 'Churn Label']).columns

Index(['Location', 'Device Class', 'Games Product', 'Music Product',
       'Education Product', 'Call Center', 'Video Product', 'Use MyApp',
       'Payment Method', 'Monthly Purchase', 'CLTV', 'CLTV per Purchase',
       'Total Service', 'Cluster'],
      dtype='object')

In [156]:
import pickle
import pandas as pd
from catboost import CatBoostRegressor

model = CatBoostRegressor()
# Pastikan untuk menginstal library CatBoost di environment tempat TabPy berjalan
model.load_model('model_cb')
# Melakukan prediksi dengan model
predictions = model.predict(df.drop(columns=['Tenure Months', 'Churn Label']))
# Mengembalikan hasil prediksi ke Tableau
predictions.tolist()

[12.150410225502023,
 6.029671262513325,
 24.69884278895352,
 37.88617652124175,
 51.163150113387594,
 17.542622101504108,
 13.695012589142301,
 5.0125148478404995,
 32.26681115492049,
 8.045602990862129,
 10.487262870993959,
 6.454386574310233,
 25.718295456958334,
 31.085333257301663,
 10.266140649833282,
 27.62008361388606,
 21.858184876437218,
 12.21296432827832,
 16.751079012695442,
 7.5248052614383205,
 14.290600449628794,
 15.55134270503321,
 5.282187093094484,
 35.05492340413558,
 60.07813323922847,
 37.74168792585094,
 7.163816202111949,
 24.319027427666114,
 7.200333194607838,
 11.041615343047237,
 50.93236987897534,
 5.959368133482398,
 4.422462118751872,
 2.2215104502785845,
 20.822086782051134,
 4.435502124543088,
 12.198051629661286,
 18.14309096067349,
 13.027654947910268,
 28.136620346742408,
 34.63257922938946,
 30.297001623680472,
 5.8289578117360215,
 19.610925645611434,
 15.500392853149163,
 15.75846554047132,
 18.933163613169725,
 5.762179242822748,
 18.93634356336

In [ ]:
# from catboost import CatBoostRegressor
# from sklearn.model_selection import GridSearchCV, train_test_split
# from sklearn.metrics import mean_absolute_error

# # Define the parameter grid
# param_grid = {
#     'depth': [4, 6, 8, 10],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'iterations': [500, 1000, 1500],
#     'l2_leaf_reg': [1, 3, 5, 7, 9]
# }

# # Define the model
# catboost_model = CatBoostRegressor(cat_features=cat_features, random_state=9999)

# # Set up the GridSearchCV
# grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, cv=3, scoring='neg_mean_absolute_error')

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

# # Perform the grid search
# grid_search.fit(X_train, y_train)

# # Get the best parameters
# best_params = grid_search.best_params_

# # Evaluate the best model
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# print('Best MAE:', mae)

# # Output the best parameters
# print('Best Parameters:', best_params)


In [ ]:
# prediksi dengan catboost untuk data df_churn_0
df_churn_0 = df[df['Churn Label']==0]

# prediksi
y_pred_0 = model.predict(df_churn_0.drop(['Tenure Months', 'Churn Label'], axis=1))

In [ ]:
df_pred = pd.DataFrame(y_pred_0)
df_pred

,0
0,11.223903
1,9.910957
2,21.585389
3,27.242998
4,8.487680
...,...
5169,14.716887
5170,23.937184
5171,47.710253
5172,8.934565


In [ ]:
# use MLP regression
from sklearn.neural_network import MLPRegressor

# ambil hanya churn label 1
df_churn = df[df['Churn Label']==1]
df_churn.drop(columns=['Churn Label'], inplace=True)

# dummies
df_churn = pd.get_dummies(df_churn, columns=['Device Class', 'Payment Method'])
df_churn = pd.get_dummies(df_churn, columns=['Location', 'Games Product',
       'Music Product', 'Education Product', 'Call Center', 'Video Product',
       'Use MyApp', 'No Internet Service'], drop_first=True)

# standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

df_scale = scaler.fit_transform(df_churn.drop(['Tenure Months'], axis=1))

# split
X = df_scale
y = df_churn['Tenure Months']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

# fit model
model = MLPRegressor(random_state=2020, hidden_layer_sizes=(200,200,200, 200, 200),
                     max_iter=1000, alpha=400, verbose=True, learning_rate='adaptive')
model.fit(X_train, y_train)

# cek akurasi training
model.score(X_train, y_train)

In [221]:
# evaluate model
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

print('MAE:', mean_absolute_error(y_test, y_pred))

# r2 scores
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print('R2:', r2)